# <center> Testing the purification technique from the paper.
## <center> Use only $\ket{0}_E$ to avoid confusions from before.

In [1]:
import sys
sys.path.append('..')
# reload local packages automatically
%load_ext autoreload
%autoreload 2

In [2]:
# Okay I am going to forget about everything else and just focus on implementing the purifcation tensor network structure
import qutip as qt
from opentn.circuits import get_unitary_adchannel
from opentn.channels import get_krauss_from_unitary
from opentn.states.qubits import up,down, plus, minus
import numpy as np

In [3]:
gamma = 0.5
U = get_unitary_adchannel(gamma=gamma)
print(U)

[[ 1.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.70710678+0.j  0.70710678+0.j]
 [ 0.        +0.j  0.        +0.j  0.70710678+0.j -0.70710678+0.j]
 [ 0.        +0.j  1.        +0.j  0.        +0.j  0.        +0.j]]


In [4]:
krauss_list = get_krauss_from_unitary(U)
krauss_list

[array([[1.        +0.j, 0.        +0.j],
        [0.        +0.j, 0.70710678+0.j]]),
 array([[0.        +0.j, 0.70710678+0.j],
        [0.        +0.j, 0.        +0.j]])]

In [22]:
# now we need to create new structure for purifcations in TN
# initial purified state
a,b = plus
psi_pure = np.zeros(shape=(1,2,1,1),dtype=np.complex128) #vL up down vR. Assuming environment is by default zero
psi_pure[0,0,0,0] = a
psi_pure[:,1,:,:] = b
psi_pure

array([[[[0.70710678+0.j]],

        [[0.70710678+0.j]]]])

In [23]:
# new purified state after krauss act on physical system
psi_end_pure = np.zeros(shape=(1,2,2,1),dtype=np.complex128) #vL up down vR. Assuming environment is by default zero
psi_end_pure[0,:,0,0] = krauss_list[0]@psi_pure[0,:,0,0]
psi_end_pure[0,:,1,0] = krauss_list[1]@psi_pure[0,:,0,0]
psi_end_pure

array([[[[0.70710678+0.j],
         [0.5       +0.j]],

        [[0.5       +0.j],
         [0.        +0.j]]]])

In [24]:
# tracing out the environment:
"""
        __|__      
       /  1   \  
    ---|0 P 3|--- 
       \__2__/    
          |      
     
        __|__     
       /  2  \    
    ---|0 P* 3|--- 
       \__1__/    
          |       
"""
rho_A = np.tensordot(psi_end_pure, psi_end_pure.conj(),axes=(2,2)) # vL up (down) vR x vL* up* (down*) vR* ->  vL up vR vL* up* vR*
np.squeeze(rho_A)

array([[0.75      +0.j, 0.35355339+0.j],
       [0.35355339+0.j, 0.25      +0.j]])

# <center> 8th february

Checkpoints: (For one site for now)
1. Generalize MPO to suit the purified structure
2. Test the generation of np.stack for K structure of 3D
3. Create function for contration:
    input: krauss list (& Purified Tensor)
    output: 4d tensor with vL, vR, P, E*s dim (makes sense to have it modify the purified tensor since it is also MPO)
4. Check by contracting with its conjugate (contract virtual dims)

In [25]:
from opentn.tensors import MPO

In [26]:
# I have to first transpose psi_pure so it fits my MPO ordering
A = MPO([psi_pure.transpose((0,3,1,2))])
A

MPO: ([array([[[[0.70710678+0.j],
         [0.70710678+0.j]]]])]) with dims (1, 1, 2, 1)

In [27]:
A.apply_krauss_operators(krauss_list=krauss_list)

In [28]:
A.get_density_matrix()

array([[0.75      +0.j, 0.35355339+0.j],
       [0.35355339+0.j, 0.25      +0.j]])

# Doing for Loop for different initial states and different gammas:

In [29]:
from opentn.channels import quantum_channel

In [30]:
gammas = np.linspace(0,1,21)

# initial state: fixed here
phys_init = plus
a,b = phys_init
psi_pure = np.zeros(shape=(1,1,2,1),dtype=np.complex128) #vL vR up down. Assuming environment is by default zero
psi_pure[:,:,0,:] = a
psi_pure[:,:,1,:] = b

for gamma in gammas:
    A = MPO([psi_pure])
    krauss_list_i = get_krauss_from_unitary(get_unitary_adchannel(gamma=gamma))
    A.apply_krauss_operators(krauss_list=krauss_list_i)
    rho_purified = A.get_density_matrix()
    rho_channel = quantum_channel(state=phys_init, krauss_list=krauss_list_i)
    print(np.allclose(rho_purified, rho_channel))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [31]:
from opentn.states.qubits import up, down, minus, plus

In [33]:
# fix gamma here
gamma = 0.5

# initial states
states = [up, down, minus, plus]

for init_state in states:
    phys_init = init_state
    a,b = phys_init
    psi_pure = np.zeros(shape=(1,1,2,1),dtype=np.complex128) #vL vR up down. Assuming environment is by default zero
    psi_pure[:,:,0,:] = a
    psi_pure[:,:,1,:] = b

    A = MPO([psi_pure])
    krauss_list_i = get_krauss_from_unitary(get_unitary_adchannel(gamma=gamma))
    A.apply_krauss_operators(krauss_list=krauss_list_i)
    rho_purified = A.get_density_matrix()
    rho_channel = quantum_channel(state=phys_init, krauss_list=krauss_list_i)
    print(np.allclose(rho_purified, rho_channel))

True
True
True
True


# 10 Feb

Todo:
- allow MPO to represent a multisite structure
    - on each site have a pure state for physical and environment
    - environment on 0 for all of them
    - create a class function that initializes this. default env to zero
    - create a function called apply_quantum_map that applies krauss operators to single site

### NOTE: i have updated the methods in the MPO class
### TODO:
1. create multi site mpo
2. apply krauss to one site
3. get reduced density matrix at the site where krauss where applied

In [38]:
# decide index on which krauss will be applied
idx = 0
# create multi site MPOP
phys_states = [plus for i in range(4)]
ms_mpo = MPO.create_purified(phys_states)
print(ms_mpo.Ws[2].shape)
# create kraus list:
krauss_list_05 = get_krauss_from_unitary(get_unitary_adchannel(gamma=0.5))
# apply it to a single site
ms_mpo.apply_krauss_operators(krauss_list=krauss_list_05, idx=idx)

for W in ms_mpo.Ws:
    print(W.shape)


(1, 1, 2, 1)
(1, 1, 2, 2)
(1, 1, 2, 1)
(1, 1, 2, 1)
(1, 1, 2, 1)


In [46]:
# should be equal to our previous result
rho = ms_mpo.get_partial_density(idx=0)
print(rho)
np.allclose(quantum_channel(state=plus, krauss_list=krauss_list_05), rho)

[[0.75      +0.j 0.35355339+0.j]
 [0.35355339+0.j 0.25      +0.j]]


True

In [49]:
# this should be unchanged:

unchanged = np.outer(plus,plus)

for i in range(4):
    if i!=idx:
        print(np.allclose(ms_mpo.get_partial_density(idx=i), unchanged))

True
True
True


In [ ]:
# NOTE:
# putting down all my thoughts. these are the functions I want to have
# 1. a function that generates another MPO (or lit of Ws) corresponding to tensoring each of the w in Ws with its adjoint and contracting the legs corresponding to the environment.
# 2. a function that takes all the Ws and contract them to get one whole single density matrix. could even be outside the class and just accept and MPO object as input parameter
# 3. a function that contracts two adjacent Ws in the MPO along the virtual leg between them.
# 4. generalize the functino contract_purified_krauss to apply_krauss operator to act on the Ws in MPO but only in one of the indexes.